In [1]:
from pymoab.core import Core  # meshdb
import pymoab.types as mt # moab types
from pydtk2 import *  # mapper
from ovcg import *  # fluid solver
from fesol import *  # FEM solver
from mpi4py import MPI  # fix openmpi with python
import numpy as np
from meshio.gmsh_io import read
from numpy import linalg as LA

In [2]:
# retrieve the interface grids and convert them into
# unstructure Q1 elements

g_f = Grids('inputs/fluid_quad.hdf')
grid_num = 0
indices = g_f.get_boundary_indices(grid=grid_num, side=1, axis=0)
# to get the geometry data, we need another instance in pyoverture
# the geometry class
gdata = Geometry(grid=grid_num, cg=g_f)
# get connectivity and vertices
conn_int_f = indices.generate_surface_connectivity(dim=2)
vert_int_f = gdata.get_vertices(indices)

In [3]:
vert_int_s, conn_int_s, _, phy_data, _ = read('inputs/solid_tri.msh')

# get the union of node ids on boundary of tag 5
temp = set()
for i in range(conn_int_s['line'].shape[0]):
    if phy_data['line']['gmsh:physical'][i] == 5:
        temp |= set(list(conn_int_s['line'][i]))

# make a list and table
temp = list(temp)
temp.sort()
vert_int_s = vert_int_s[temp] # vertices
table = {}
for i in range(len(temp)):
    table[temp[i]] = i
temp = []
for i in range(conn_int_s['line'].shape[0]):
    if phy_data['line']['gmsh:physical'][i] == 5:
        temp.append([])
        for j in conn_int_s['line'][i]:
            temp[-1].append(table[j])
conn_int_s = temp

In [4]:
# convert integers to uint64 for both fluid and solid, this is for moab
conn_int_f = np.array(conn_int_f, dtype=np.uint64)
conn_int_s = np.array(conn_int_s, dtype=np.uint64)

# also, convert fluid vertices to 3D
temp = np.zeros(shape=(vert_int_f.shape[0], 3))
temp[:, 0:2] = vert_int_f
vert_int_f = temp

In [5]:
# fluid interface meshdb
mdb_int_f = Core()
vert_int_f_range = mdb_int_f.create_vertices(vert_int_f.flatten())
conn_int_f[:] += vert_int_f_range[0] # adjust connectivity, for this case, just add 1
mdb_int_f.create_elements(mt.MBEDGE, conn_int_f)

# tags, where T for temperature and F for flux
tag_int_f = mdb_int_f.tag_get_handle(
    'intTf',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_f = np.zeros(vert_int_f.shape[0])
mdb_int_f.tag_set_data(tag_int_f, vert_int_f_range, T_int_f)
flux_tag_f = mdb_int_f.tag_get_handle(
    'intFf', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_f = np.zeros(vert_int_f.shape[0])
mdb_int_f.tag_set_data(flux_tag_f, vert_int_f_range, flux_int_f)

In [6]:
# solid interface meshdb
mdb_int_s = Core()
vert_int_s_range = mdb_int_s.create_vertices(vert_int_s.flatten())
conn_int_s[:] += vert_int_s_range[0]
mdb_int_s.create_elements(mt.MBEDGE, conn_int_s)

# tags, where T for temperature and F for flux
tag_int_s = mdb_int_s.tag_get_handle(
    'intTs',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_s = np.zeros(vert_int_s.shape[0])
mdb_int_s.tag_set_data(tag_int_s, vert_int_s_range, T_int_s)
flux_tag_s = mdb_int_s.tag_get_handle(
    'intFs', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_s = np.zeros(vert_int_s.shape[0])
mdb_int_s.tag_set_data(flux_tag_s, vert_int_s_range, flux_int_s)

In [7]:
# configure mappers, DTK uses RBFs, the radius should not be small
# otherwise, you may not have enough neighborhood for the fitting
radius = 0.4

# create solid manager
dtk_mnr_s = DTK2MoabManager(mdb_int_s)
dtk_mnr_s.register_tags(['intTs', 'intFs'])

# create fluid manager
dtk_mnr_f = DTK2MoabManager(mdb_int_f)
dtk_mnr_f.register_tags(['intTf', 'intFf'])

# solid temperature to fluid mapper
sT2f_mapper = DTK2Mapper(source=dtk_mnr_s, target=dtk_mnr_f, r=radius)
sT2f_mapper.register_coupled_tags('intTs', 'intTf')

# fluid flux to solid mapper
fF2s_mapper = DTK2Mapper(source=dtk_mnr_f, target=dtk_mnr_s, r=radius)
fF2s_mapper.register_coupled_tags('intFf', 'intFs')

In [8]:
# FEM solver
kappa_s = 0.7
specific_heat = 0.485
solver_s = HeatSolver(kappa=kappa_s, c=specific_heat)
solver_s.load_mesh('inputs/solid_tri.msh')
solver_s.set_initial_condition(0.0)
solver_s.set_boundary(2, 0.0, boundary_type='Neumann') # bot
solver_s.set_boundary(3, 1.0) # right
solver_s.set_boundary(4, 0.0, boundary_type='Neumann') # top
solver_s.set_boundary(5, 0.0, boundary_type='Neumann') # interface
solver_s.set_source(0.0) # forcing term

# init
solver_s.init_solver()

solver_s.output_file = 'outputs/solid.pvd'

In [9]:
# fluid solver
solver_f = Cgins()
solver_f.read_cmd('inputs/fluidCgins.cmd')
solver_f.init_grid()
solver_f.init_temperature_interface(grid=grid_num, side=1, axis=0)  # must do this!
solver_f.init_solver(debug=0)

kappa_f = solver_f.get_thermal_conductivity()
assert kappa_f > 0.0

In [10]:
inall = g_f.get_indices(0)
vertsall = gdata.get_vertices(inall)
connall = inall.generate_volume_connectivity(2)
temp = np.zeros(shape=(vertsall.shape[0],3), dtype=np.float64)
temp[:,0:2] = vertsall
vertsall = temp
mdbF = Core()
vr = mdbF.create_vertices(vertsall.flatten())
connall[:] += vr[0]
connall = np.array(connall, dtype=np.uint64)
mdbF.create_elements(mt.MBQUAD, connall)
tagF = mdbF.tag_get_handle(
    'fT',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True
)

In [11]:
# coupling
Tfinal = 1.0
N = 200
dt = Tfinal / N
t = 0.0
tfprevInt = np.zeros_like(T_int_f)
omega = 0.5 # relaxation parameter

In [12]:
for i in range(N):
    t += dt
    solver_f.backup_current_solutions()
    solver_s.backup()
    
    count = 0
    
    while 1:
        count = count + 1
        tfprevInt[:] = T_int_f

    ##########
    # FLUID I
    ##########
    
    # advance
        solver_f.advance(t, dt)
    # get fluid interface heat flux
        flux_int_f[:] = solver_f.get_normal_heat_flux_interface(grid=grid_num, side=1, axis=0)
    # scale
        flux_int_f[:] *= kappa_f

    ###########
    # MAPPING
    ###########
    
    # put to meshdb
        mdb_int_f.tag_set_data(flux_tag_f, vert_int_f_range, flux_int_f)
    # transfer
        fF2s_mapper.apply('intFf', 'intFs')

    ##########
    # SOLID
    ##########
    
    # get data from meshdb
        flux_int_s[:] = mdb_int_s.tag_get_data(flux_tag_s, vert_int_s_range, True)
    # solid update the heat flux interface, remind the tag is 5
    # cht_interface indicates that this value is given by CHT interface
    # so internally, it will take care of scaling and normal directions
        solver_s.update_flux_interface(5, flux_int_s, cht_interface=True)
    # advance
        solver_s.advance(dt, t)
    # get temperature
        T_int_s[:] = solver_s.get_solutions(where='boundary', tag=5)

    ###########
    # MAPPING
    ###########
    
    # put to meshdb
        mdb_int_s.tag_set_data(tag_int_s, vert_int_s_range, T_int_s)
    # transfer
        sT2f_mapper.apply('intTs', 'intTf')

    ##########
    # FLUID II
    ##########
    
    # get data from meshdb
        T_int_f[:] = mdb_int_f.tag_get_data(tag_int_f, vert_int_f_range, True)
        T_int_f[:] = (1.0-omega)*T_int_f+omega*tfprevInt
    # fluid updating the temperature boundary condition
        solver_f.put_temperature_interface(0, 1, 0, T_int_f)
        
        err = T_int_f-tfprevInt
        err = LA.norm(err) # TODO should be relative error right?
        if err < 1e-3 or count >= 100:
            break
        else:
            solver_f.restore_previous_solutions()
            solver_s.restore()
    
    ##########
    # OUTPUTS
    ##########

    print('error=%f, step=%i, iterations=%i' % (err, i, count))
    solver_s.write()
    fT = solver_f.get_temperature(0, inall)
    mdbF.tag_set_data(tagF, vr, fT)
    mdbF.write_file('outputs/fluid%i.vtk' % i)

error=0.000000, step=0, iterations=1
error=0.000002, step=1, iterations=1
error=0.000017, step=2, iterations=1
error=0.000113, step=3, iterations=1
error=0.000525, step=4, iterations=1
error=0.000229, step=5, iterations=2
error=0.000566, step=6, iterations=2
error=0.000154, step=7, iterations=3
error=0.000250, step=8, iterations=3
error=0.000348, step=9, iterations=3
error=0.000447, step=10, iterations=3
error=0.000533, step=11, iterations=3
error=0.000610, step=12, iterations=3
error=0.000673, step=13, iterations=3
error=0.000726, step=14, iterations=3
error=0.000767, step=15, iterations=3
error=0.000799, step=16, iterations=3
error=0.000822, step=17, iterations=3
error=0.000838, step=18, iterations=3
error=0.000848, step=19, iterations=3
error=0.000853, step=20, iterations=3
error=0.000853, step=21, iterations=3
error=0.000850, step=22, iterations=3
error=0.000844, step=23, iterations=3
error=0.000835, step=24, iterations=3
error=0.000825, step=25, iterations=3
error=0.000813, step=2